## Latent Dirichlet Allocation
http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation/

## Code

https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

## Data loading

In [5]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv')
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [6]:
documents = data[['headline_text']]
documents['index'] = documents.index
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [10]:
print(f"Total number of documents: {len(documents)}")
documents.head(5)

Total number of documents: 1186018


,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Data Preprocessing

In [17]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2020)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/john/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
def lemmatize_stemming(token):
    lem_token = WordNetLemmatizer().lemmatize(token, pos='v')
    stemmer = SnowballStemmer(language='english')
    return stemmer.stem(lem_token)

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [48]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print(f"Original document: \n{doc_sample.split()}\n")
print(f"Tokenized and lemmatized document: \n{preprocess(doc_sample)}")

Original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']

Tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [49]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs.head()

0     [decid, communiti, broadcast, licenc]
1                        [wit, awar, defam]
2    [call, infrastructur, protect, summit]
3               [staff, aust, strike, rise]
4      [strike, affect, australian, travel]
Name: headline_text, dtype: object

## Bag of Words on the Data set

Create a dictionary from 'processed_docs' containing the number of times a word appears in the training set.

In [51]:
dictionary = gensim.corpora.Dictionary(processed_docs)

print(f"There are {len(dictionary)} words in total")

count = 0
for k,v in dictionary.items():
    print(k,v)
    count += 1
    if count > 10:
        break

There are 67118 words in total
0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


Filter out tokens that appear in
* less than 15 documents (absolute number) or
* more than 0.5 documents (fraction of total corpus size, not absolute number).
* after the above two steps, keep only the first 100000 most frequent tokens.

In [57]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

print(f"There are {len(dictionary)} words in filtered dictionary in total")

There are 14939 words in filtered dictionary in total


## Gensim doc2bow

In [68]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(f"Words and frequencies in the sentence:\n{bow_corpus[4310]}")
sent = []
for word_num, freq in bow_corpus[4310]:
    sent.append(dictionary[word_num])
print(sent)

Words and frequencies in the sentence:
[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3567, 1), (3568, 1)]
['govt', 'group', 'vote', 'local', 'want', 'compulsori', 'ratepay']


## TF-IDF

In [71]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

corpus_tfidf[4310]

[(162, 0.25617525269671065),
 (240, 0.3011111395538523),
 (292, 0.33416888830557095),
 (589, 0.33377677352466983),
 (838, 0.3121925622107832),
 (3567, 0.5158075532653446),
 (3568, 0.5070590825348879)]

## Running LDA using Bag of Words

In [83]:
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=10,
                                       id2word=dictionary,
                                       passes=5,
                                       workers=2)

In [84]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic {} \nWords: {}".format(idx, topic))

Topic 0 
Words: 0.027*"home" + 0.023*"live" + 0.020*"hous" + 0.016*"bank" + 0.015*"guilti" + 0.015*"perth" + 0.014*"return" + 0.011*"care" + 0.011*"violenc" + 0.010*"white"
Topic 1 
Words: 0.032*"elect" + 0.025*"kill" + 0.018*"coast" + 0.014*"tasmanian" + 0.013*"attack" + 0.013*"say" + 0.012*"victoria" + 0.012*"deal" + 0.012*"gold" + 0.010*"state"
Topic 2 
Words: 0.035*"court" + 0.026*"news" + 0.023*"face" + 0.014*"game" + 0.014*"student" + 0.013*"tell" + 0.012*"polit" + 0.012*"review" + 0.012*"announc" + 0.011*"famili"
Topic 3 
Words: 0.024*"melbourn" + 0.020*"die" + 0.019*"canberra" + 0.017*"health" + 0.015*"hospit" + 0.015*"royal" + 0.015*"rural" + 0.014*"crash" + 0.013*"interview" + 0.013*"chang"
Topic 4 
Words: 0.048*"polic" + 0.028*"charg" + 0.023*"murder" + 0.019*"woman" + 0.019*"donald" + 0.016*"shoot" + 0.016*"alleg" + 0.014*"miss" + 0.012*"trial" + 0.012*"arrest"
Topic 5 
Words: 0.024*"market" + 0.023*"south" + 0.018*"protest" + 0.016*"farmer" + 0.015*"australia" + 0.015*"fig

In [74]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,
                                       num_topics=10,
                                       id2word=dictionary,
                                       passes=2,
                                       workers=4)

In [75]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic {} \nWords: {}".format(idx, topic))

Topic 0 
Words: 0.022*"news" + 0.009*"queensland" + 0.009*"north" + 0.009*"climat" + 0.008*"australia" + 0.008*"south" + 0.007*"drought" + 0.007*"korea" + 0.007*"scott" + 0.006*"christma"
Topic 1 
Words: 0.017*"rural" + 0.015*"govern" + 0.009*"health" + 0.007*"monday" + 0.007*"nation" + 0.007*"thursday" + 0.006*"care" + 0.005*"financ" + 0.005*"violenc" + 0.005*"fund"
Topic 2 
Words: 0.014*"donald" + 0.011*"final" + 0.009*"australia" + 0.009*"leagu" + 0.008*"world" + 0.007*"sport" + 0.007*"cricket" + 0.006*"rugbi" + 0.006*"open" + 0.006*"video"
Topic 3 
Words: 0.012*"market" + 0.009*"wednesday" + 0.008*"share" + 0.008*"wall" + 0.007*"australian" + 0.006*"street" + 0.006*"brexit" + 0.005*"coal" + 0.005*"facebook" + 0.005*"toni"
Topic 4 
Words: 0.012*"royal" + 0.011*"stori" + 0.009*"commiss" + 0.009*"turnbul" + 0.008*"michael" + 0.007*"grandstand" + 0.007*"mother" + 0.007*"juli" + 0.007*"sentenc" + 0.006*"jail"
Topic 5 
Words: 0.010*"elect" + 0.007*"labor" + 0.007*"liber" + 0.006*"say" + 

## Performance evaluation by classifying sample document using LDA Bag of Words model

In [85]:
sample_id = 4310
print(processed_docs[sample_id])

bow_corpus_sorted = sorted(lda_model[bow_corpus[sample_id]], key=lambda tup: -1*tup[1])
for index, score in bow_corpus_sorted:
    print("\nScore: {} \nTopic: {}".format(score, lda_model.print_topic(index,10)))
    
    

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

Score: 0.6377481818199158 
Topic: 0.024*"govern" + 0.017*"plan" + 0.015*"feder" + 0.014*"tasmania" + 0.014*"water" + 0.012*"council" + 0.012*"fund" + 0.012*"indigen" + 0.010*"call" + 0.010*"communiti"

Score: 0.26220080256462097 
Topic: 0.024*"market" + 0.023*"south" + 0.018*"protest" + 0.016*"farmer" + 0.015*"australia" + 0.015*"fight" + 0.015*"busi" + 0.015*"rise" + 0.014*"price" + 0.013*"fall"

Score: 0.012509556487202644 
Topic: 0.032*"elect" + 0.025*"kill" + 0.018*"coast" + 0.014*"tasmanian" + 0.013*"attack" + 0.013*"say" + 0.012*"victoria" + 0.012*"deal" + 0.012*"gold" + 0.010*"state"

Score: 0.012506111524999142 
Topic: 0.031*"australia" + 0.025*"nation" + 0.022*"north" + 0.021*"test" + 0.018*"island" + 0.014*"west" + 0.013*"drum" + 0.013*"stori" + 0.012*"scott" + 0.012*"morrison"

Score: 0.012506071478128433 
Topic: 0.048*"polic" + 0.028*"charg" + 0.023*"murder" + 0.019*"woman" + 0.019*"donald" + 0.016*"shoot"

## Performance evaluation by classifying sample document using LDA TF-IDF model

In [79]:
sample_id = 4310
print(processed_docs[sample_id])

bow_corpus_sorted = sorted(lda_model_tfidf[bow_corpus[sample_id]], key=lambda tup: -1*tup[1])
for index, score in bow_corpus_sorted:
    print("\nScore: {} \nTopic: {}".format(score, lda_model_tfidf.print_topic(index,10)))
    
    

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

Score: 0.6073325276374817 
Topic: 0.010*"elect" + 0.007*"labor" + 0.007*"liber" + 0.006*"say" + 0.006*"council" + 0.006*"plan" + 0.005*"peter" + 0.005*"senat" + 0.005*"august" + 0.005*"green"

Score: 0.292629212141037 
Topic: 0.012*"interview" + 0.011*"polic" + 0.009*"death" + 0.009*"shoot" + 0.007*"attack" + 0.006*"john" + 0.005*"inquest" + 0.005*"septemb" + 0.005*"suspect" + 0.005*"hunt"

Score: 0.012505852617323399 
Topic: 0.017*"rural" + 0.015*"govern" + 0.009*"health" + 0.007*"monday" + 0.007*"nation" + 0.007*"thursday" + 0.006*"care" + 0.005*"financ" + 0.005*"violenc" + 0.005*"fund"

Score: 0.01250543911010027 
Topic: 0.012*"market" + 0.009*"wednesday" + 0.008*"share" + 0.008*"wall" + 0.007*"australian" + 0.006*"street" + 0.006*"brexit" + 0.005*"coal" + 0.005*"facebook" + 0.005*"toni"

Score: 0.01250492874532938 
Topic: 0.012*"charg" + 0.010*"guilti" + 0.010*"alleg" + 0.010*"murder" + 0.009*"polic" + 0.009*"assa

## Testing model on unseen document

In [86]:
unseen_doc = 'Today programming skills are required in many industries. Tech companies hire as never before.'
bow_vector = dictionary.doc2bow(preprocess(unseen_doc))

bow_vector_sorted = sorted(lda_model[bow_vector], key=lambda tup:-1*tup[1])
for index, score in bow_vector_sorted:
    print("Score: {} \nTopic: {}".format(score, lda_model.print_topic(index, 5)))
    
    

Score: 0.39934155344963074 
Topic: 0.035*"court" + 0.026*"news" + 0.023*"face" + 0.014*"game" + 0.014*"student"
Score: 0.1457478106021881 
Topic: 0.032*"elect" + 0.025*"kill" + 0.018*"coast" + 0.014*"tasmanian" + 0.013*"attack"
Score: 0.1422124207019806 
Topic: 0.024*"govern" + 0.017*"plan" + 0.015*"feder" + 0.014*"tasmania" + 0.014*"water"
Score: 0.13493379950523376 
Topic: 0.024*"market" + 0.023*"south" + 0.018*"protest" + 0.016*"farmer" + 0.015*"australia"
Score: 0.1221773698925972 
Topic: 0.040*"year" + 0.025*"china" + 0.019*"jail" + 0.018*"bushfir" + 0.017*"death"
Score: 0.011119470000267029 
Topic: 0.031*"australia" + 0.025*"nation" + 0.022*"north" + 0.021*"test" + 0.018*"island"
Score: 0.011118164286017418 
Topic: 0.024*"melbourn" + 0.020*"die" + 0.019*"canberra" + 0.017*"health" + 0.015*"hospit"
Score: 0.011117043904960155 
Topic: 0.042*"australian" + 0.037*"trump" + 0.024*"world" + 0.023*"queensland" + 0.017*"adelaid"
Score: 0.011116147972643375 
Topic: 0.027*"home" + 0.023*"l

In [82]:
unseen_doc = 'Today programming skills are required in many industries. Tech companies hire as never before.'
bow_vector = dictionary.doc2bow(preprocess(unseen_doc))

bow_vector_sorted = sorted(lda_model_tfidf[bow_vector], key=lambda tup:-1*tup[1])
for index, score in bow_vector_sorted:
    print("Score: {} \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))
    
    

Score: 0.328738272190094 
Topic: 0.017*"rural" + 0.015*"govern" + 0.009*"health" + 0.007*"monday" + 0.007*"nation"
Score: 0.1798470914363861 
Topic: 0.019*"countri" + 0.014*"hour" + 0.014*"drum" + 0.012*"live" + 0.010*"weather"
Score: 0.17456398904323578 
Topic: 0.010*"elect" + 0.007*"labor" + 0.007*"liber" + 0.006*"say" + 0.006*"council"
Score: 0.13153955340385437 
Topic: 0.029*"trump" + 0.019*"crash" + 0.011*"die" + 0.010*"woman" + 0.008*"fatal"
Score: 0.12972453236579895 
Topic: 0.012*"charg" + 0.010*"guilti" + 0.010*"alleg" + 0.010*"murder" + 0.009*"polic"
Score: 0.011117811314761639 
Topic: 0.012*"market" + 0.009*"wednesday" + 0.008*"share" + 0.008*"wall" + 0.007*"australian"
Score: 0.011117374524474144 
Topic: 0.022*"news" + 0.009*"queensland" + 0.009*"north" + 0.009*"climat" + 0.008*"australia"
Score: 0.01111723855137825 
Topic: 0.012*"royal" + 0.011*"stori" + 0.009*"commiss" + 0.009*"turnbul" + 0.008*"michael"
Score: 0.011117043904960155 
Topic: 0.012*"interview" + 0.011*"polic